### Week 5 Lab Exercise Notebook Template

This is the notebook template for week 5 lab exercise. In this exercise, you are asked to find out the top 5 movies of a given genre based on the number of ratings a move receives. You should print the movie ids and the number of ratings received on the console.

The following cell import necessary packages and expose pyspark to Jupyter Notebook.

In [1]:
import findspark
from ml_utils import *
findspark.init()
from pyspark import SparkContext

<span style="color:teal; font-weight: bold">The following cell  creates a new SparkContext. It also specify the input path. Again, you can change the input path to match your own path.You can also change the genre value</span> 

In [2]:
sc = SparkContext(appName="Top Movie in Given Genra")

genre = 'Documentary'
input_path = 'hdfs://soit-hdp-pro-1.ucc.usyd.edu.au/share/movie/small/'


In [3]:
#Read input into RDDs
ratings = sc.textFile(input_path + "ratings.csv")
movieData = sc.textFile(input_path + "movies.csv")
#Write your own transformations/actions here to produce the result


<span style="color:green; font-weight: bold">In the following cell, define a few functions that you will need to implement the processing</span>

Function defined here can access global variables such as ```genre```. You will probably need functions that can check if a movie belongs to a particular genre, you will also probably need a function to extract all ratings beloning to a particular genre.

In [9]:
#Define a few functions here 
# This function convert entries of movies.csv into key,value pair of the following format
# movieID -> genre
# since there may be multiple genre per movie, this function returns a list of key,value pair
def pairMovieToGenre(record):
    try:
        movieID, title, genreList = record.strip().split(",")
        genres = genreList.strip().split("|")
        return [(movieID, (title, genre.strip())) for genre in genres]
    except:
        return []

# This function convert entries of ratings.csv into key,value pair of the following format
# movieID -> 1
def extractRating(record):
    try:
        userID, movieID, rating, timestamp = record.strip().split(",")
        return (movieID.strip(), 1)
    except:
        return ()

# This function is used by reduceByKey function to merge count of the same key
# This functions takes in two values - merged count from previous call of sumRatingCount, and the currently processed count
def sumRatingCount(reducedCount, currentCount):
    return reducedCount+currentCount

# This functions convert tuples of ((title, genre), number of rating) into key,value pair of the following format
# genre -> (title, number of rating)
def mapToPair(record):
    titleGenre, count = record
    title, genre = titleGenre
    return (genre, (title, count))

# This functions is used to calculate the top 5 movies per genre
# The input of this function is in the form of (genre, [(title, number of rating)+])
def naiveTopMovies(record):
    genre, titleRatingList = record
      # Sorted function takes in an iterator (such as list) and returns back a sorted list.
      # The key argument is an optional argument used to choose custom key for sorting.
      # In this case, since the key is the number of rating, we needed to use the lambda function to 'present' the number of rating
      # as key rather than the title.
    sortedRatingList = sorted(titleRatingList, key=lambda rec:rec[-1], reverse=True)
    return genre, sortedRatingList[:5]

<span style="color:teal; font-weight: bold">In the following cell, read the two input files as RDD and use a series of transformations/actions to extract and aggregate information from the input file to produce the required output</span>

In [4]:
movieRatings = ratings.map(extractRating)
movieRatings.collect()

[('16', 4.0),
 ('24', 1.5),
 ('32', 4.0),
 ('47', 4.0),
 ('50', 4.0),
 ('110', 4.0),
 ('150', 3.0),
 ('161', 4.0),
 ('165', 3.0),
 ('204', 0.5),
 ('223', 4.0),
 ('256', 0.5),
 ('260', 4.5),
 ('261', 1.5),
 ('277', 0.5),
 ('296', 4.0),
 ('318', 4.0),
 ('349', 4.5),
 ('356', 3.0),
 ('377', 2.5),
 ('380', 3.0),
 ('457', 4.0),
 ('480', 3.5),
 ('527', 4.5),
 ('589', 3.5),
 ('590', 3.5),
 ('592', 2.5),
 ('593', 5.0),
 ('597', 3.0),
 ('608', 3.5),
 ('648', 3.5),
 ('719', 0.5),
 ('724', 3.5),
 ('736', 3.0),
 ('780', 3.5),
 ('858', 5.0),
 ('912', 5.0),
 ('968', 4.5),
 ('1061', 4.0),
 ('1089', 4.5),
 ('1136', 5.0),
 ('1196', 4.5),
 ('1198', 4.0),
 ('1210', 4.5),
 ('1213', 5.0),
 ('1220', 4.0),
 ('1221', 5.0),
 ('1222', 5.0),
 ('1233', 4.5),
 ('1243', 3.5),
 ('1258', 4.5),
 ('1265', 3.0),
 ('1267', 4.0),
 ('1270', 3.0),
 ('1287', 4.5),
 ('1580', 3.5),
 ('1617', 3.5),
 ('1721', 1.5),
 ('1923', 1.5),
 ('1961', 3.0),
 ('2021', 4.0),
 ('2028', 4.5),
 ('2105', 1.5),
 ('2161', 1.5),
 ('2194', 4.5),
 ('

In [10]:
movieRatingsCount = ratings.map(extractRating).reduceByKey(sumRatingCount)
movieRatingsCount.collect()  

[('16', 84),
 ('24', 38),
 ('50', 228),
 ('110', 248),
 ('150', 197),
 ('161', 107),
 ('204', 34),
 ('223', 101),
 ('256', 49),
 ('277', 39),
 ('296', 325),
 ('356', 311),
 ('380', 184),
 ('457', 244),
 ('590', 201),
 ('608', 201),
 ('648', 173),
 ('724', 51),
 ('736', 141),
 ('780', 205),
 ('912', 125),
 ('1089', 131),
 ('1136', 154),
 ('1196', 228),
 ('1198', 224),
 ('1210', 222),
 ('1213', 135),
 ('1220', 99),
 ('1221', 140),
 ('1222', 111),
 ('1243', 22),
 ('1617', 137),
 ('2028', 187),
 ('2105', 40),
 ('2194', 81),
 ('2407', 62),
 ('2455', 49),
 ('2467', 28),
 ('2762', 191),
 ('2959', 207),
 ('3578', 164),
 ('4027', 92),
 ('4262', 49),
 ('4306', 158),
 ('4993', 189),
 ('5349', 102),
 ('5378', 68),
 ('5952', 167),
 ('6711', 72),
 ('6807', 46),
 ('8825', 1),
 ('33493', 54),
 ('33794', 101),
 ('45950', 11),
 ('48516', 87),
 ('48780', 72),
 ('49272', 80),
 ('52973', 31),
 ('54286', 70),
 ('1', 232),
 ('14', 25),
 ('17', 100),
 ('112', 51),
 ('376', 48),
 ('637', 22),
 ('802', 72),
 ('

In [12]:
movieGenre = movieData.flatMap(pairMovieToGenre)
movieGenre.collect()

[('1', ('Toy Story (1995)', 'Adventure')),
 ('1', ('Toy Story (1995)', 'Animation')),
 ('1', ('Toy Story (1995)', 'Children')),
 ('1', ('Toy Story (1995)', 'Comedy')),
 ('1', ('Toy Story (1995)', 'Fantasy')),
 ('2', ('Jumanji (1995)', 'Adventure')),
 ('2', ('Jumanji (1995)', 'Children')),
 ('2', ('Jumanji (1995)', 'Fantasy')),
 ('3', ('Grumpier Old Men (1995)', 'Comedy')),
 ('3', ('Grumpier Old Men (1995)', 'Romance')),
 ('4', ('Waiting to Exhale (1995)', 'Comedy')),
 ('4', ('Waiting to Exhale (1995)', 'Drama')),
 ('4', ('Waiting to Exhale (1995)', 'Romance')),
 ('5', ('Father of the Bride Part II (1995)', 'Comedy')),
 ('6', ('Heat (1995)', 'Action')),
 ('6', ('Heat (1995)', 'Crime')),
 ('6', ('Heat (1995)', 'Thriller')),
 ('7', ('Sabrina (1995)', 'Comedy')),
 ('7', ('Sabrina (1995)', 'Romance')),
 ('8', ('Tom and Huck (1995)', 'Adventure')),
 ('8', ('Tom and Huck (1995)', 'Children')),
 ('9', ('Sudden Death (1995)', 'Action')),
 ('10', ('GoldenEye (1995)', 'Action')),
 ('10', ('Golden

In [13]:
genreRatings = movieGenre.join(movieRatingsCount).values().map(mapToPair)
genreRatings.collect()

[('Comedy', ('Waiting to Exhale (1995)', 11)),
 ('Drama', ('Waiting to Exhale (1995)', 11)),
 ('Romance', ('Waiting to Exhale (1995)', 11)),
 ('Action', ('GoldenEye (1995)', 135)),
 ('Adventure', ('GoldenEye (1995)', 135)),
 ('Thriller', ('GoldenEye (1995)', 135)),
 ('Comedy', ('Dracula: Dead and Loving It (1995)', 20)),
 ('Horror', ('Dracula: Dead and Loving It (1995)', 20)),
 ('Crime', ('Casino (1995)', 84)),
 ('Drama', ('Casino (1995)', 84)),
 ('Action', ('Money Train (1995)', 16)),
 ('Comedy', ('Money Train (1995)', 16)),
 ('Crime', ('Money Train (1995)', 16)),
 ('Drama', ('Money Train (1995)', 16)),
 ('Thriller', ('Money Train (1995)', 16)),
 ('Drama', ('Powder (1995)', 38)),
 ('Sci-Fi', ('Powder (1995)', 38)),
 ('Drama', ('Othello (1995)', 14)),
 ('Action', ('Mortal Kombat (1995)', 53)),
 ('Adventure', ('Mortal Kombat (1995)', 53)),
 ('Fantasy', ('Mortal Kombat (1995)', 53)),
 ('Adventure', ('Lamerica (1994)', 2)),
 ('Drama', ('Lamerica (1994)', 2)),
 ('Drama', ('Home for the Hol

In [14]:
genreMovies = genreRatings.groupByKey(1)
genreMovies.collect()

[('Comedy', <pyspark.resultiterable.ResultIterable at 0x7fb72acc1860>),
 ('Drama', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8c88>),
 ('Romance', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8b38>),
 ('Action', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8b00>),
 ('Adventure', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8c50>),
 ('Thriller', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8e10>),
 ('Horror', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8a58>),
 ('Crime', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8f60>),
 ('Sci-Fi', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8f98>),
 ('Fantasy', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8e80>),
 ('Documentary', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8f28>),
 ('War', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8b70>),
 ('Mystery', <pyspark.resultiterable.ResultIterable at 0x7fb72acb8a20>),
 ('IMAX', <pyspark.resultiterable.ResultIterable at 0x7f

In [15]:
genreTop5Movies = genreMovies.map(naiveTopMovies)
genreTop5Movies.collect()

[('Comedy',
  [('Pulp Fiction (1994)', 325),
   ('Forrest Gump (1994)', 311),
   ('Toy Story (1995)', 232),
   ('Back to the Future (1985)', 213),
   ('Fargo (1996)', 201)]),
 ('Drama',
  [('Pulp Fiction (1994)', 325),
   ('Forrest Gump (1994)', 311),
   ('Braveheart (1995)', 248),
   ("Schindler's List (1993)", 248),
   ('American Beauty (1999)', 216)]),
 ('Romance',
  [('Forrest Gump (1994)', 311),
   ('American Beauty (1999)', 216),
   ('Speed (1994)', 197),
   ('True Lies (1994)', 184),
   ('Shrek (2001)', 158)]),
 ('Action',
  [('Jurassic Park (1993)', 294),
   ('Star Wars: Episode IV - A New Hope (1977)', 273),
   ('Terminator 2: Judgment Day (1991)', 253),
   ('Braveheart (1995)', 248),
   ('Star Wars: Episode V - The Empire Strikes Back (1980)', 228)]),
 ('Adventure',
  [('Jurassic Park (1993)', 294),
   ('Star Wars: Episode IV - A New Hope (1977)', 273),
   ('Toy Story (1995)', 232),
   ('Star Wars: Episode V - The Empire Strikes Back (1980)', 228),
   ('Raiders of the Lost Ar

In [5]:
movieGenre = movieData.flatMap(pairMovieToGenre) # we use flatMap as there are multiple genre per movie
movieGenre.collect()

[('1', 'Adventure'),
 ('1', 'Animation'),
 ('1', 'Children'),
 ('1', 'Comedy'),
 ('1', 'Fantasy'),
 ('2', 'Adventure'),
 ('2', 'Children'),
 ('2', 'Fantasy'),
 ('3', 'Comedy'),
 ('3', 'Romance'),
 ('4', 'Comedy'),
 ('4', 'Drama'),
 ('4', 'Romance'),
 ('5', 'Comedy'),
 ('6', 'Action'),
 ('6', 'Crime'),
 ('6', 'Thriller'),
 ('7', 'Comedy'),
 ('7', 'Romance'),
 ('8', 'Adventure'),
 ('8', 'Children'),
 ('9', 'Action'),
 ('10', 'Action'),
 ('10', 'Adventure'),
 ('10', 'Thriller'),
 ('11', 'Comedy'),
 ('11', 'Drama'),
 ('11', 'Romance'),
 ('12', 'Comedy'),
 ('12', 'Horror'),
 ('13', 'Adventure'),
 ('13', 'Animation'),
 ('13', 'Children'),
 ('14', 'Drama'),
 ('15', 'Action'),
 ('15', 'Adventure'),
 ('15', 'Romance'),
 ('16', 'Crime'),
 ('16', 'Drama'),
 ('17', 'Drama'),
 ('17', 'Romance'),
 ('18', 'Comedy'),
 ('19', 'Comedy'),
 ('20', 'Action'),
 ('20', 'Comedy'),
 ('20', 'Crime'),
 ('20', 'Drama'),
 ('20', 'Thriller'),
 ('21', 'Comedy'),
 ('21', 'Crime'),
 ('21', 'Thriller'),
 ('22', 'Crime'

In [6]:
genreRatings = movieGenre.join(movieRatings).values()
genreRatings.collect()    

[('Comedy', 3.5),
 ('Comedy', 3.0),
 ('Comedy', 4.0),
 ('Comedy', 4.0),
 ('Comedy', 3.0),
 ('Comedy', 3.0),
 ('Comedy', 3.0),
 ('Comedy', 1.5),
 ('Comedy', 3.0),
 ('Comedy', 1.0),
 ('Comedy', 2.0),
 ('Drama', 3.5),
 ('Drama', 3.0),
 ('Drama', 4.0),
 ('Drama', 4.0),
 ('Drama', 3.0),
 ('Drama', 3.0),
 ('Drama', 3.0),
 ('Drama', 1.5),
 ('Drama', 3.0),
 ('Drama', 1.0),
 ('Drama', 2.0),
 ('Romance', 3.5),
 ('Romance', 3.0),
 ('Romance', 4.0),
 ('Romance', 4.0),
 ('Romance', 3.0),
 ('Romance', 3.0),
 ('Romance', 3.0),
 ('Romance', 1.5),
 ('Romance', 3.0),
 ('Romance', 1.0),
 ('Romance', 2.0),
 ('Action', 4.0),
 ('Action', 3.0),
 ('Action', 3.0),
 ('Action', 3.0),
 ('Action', 4.0),
 ('Action', 5.0),
 ('Action', 3.0),
 ('Action', 5.0),
 ('Action', 2.5),
 ('Action', 3.5),
 ('Action', 4.0),
 ('Action', 3.0),
 ('Action', 2.0),
 ('Action', 4.0),
 ('Action', 3.0),
 ('Action', 4.0),
 ('Action', 4.0),
 ('Action', 3.0),
 ('Action', 3.5),
 ('Action', 3.0),
 ('Action', 4.0),
 ('Action', 3.0),
 ('Action'

In [7]:
genreRatingsAverage = genreRatings.aggregateByKey((0.0,0), mergeRating, mergeCombiners, 1).map(mapAverageRating)
genreRatingsAverage.collect()    

[('Comedy', 3.4209959269478385),
 ('Drama', 3.6502661839863713),
 ('Romance', 3.544254739708809),
 ('Action', 3.4514500881269026),
 ('Adventure', 3.518027387762177),
 ('Thriller', 3.4955613220431574),
 ('Horror', 3.281097331830139),
 ('Crime', 3.6423924334372098),
 ('Sci-Fi', 3.4544805001488537),
 ('Fantasy', 3.5004591789879695),
 ('Mystery', 3.652043269230769),
 ('Children', 3.4394294887626575),
 ('Documentary', 3.6430348258706466),
 ('War', 3.7832017844886754),
 ('IMAX', 3.641820580474934),
 ('Musical', 3.57196174480989),
 ('Western', 3.565687121866897),
 ('Animation', 3.6353503184713376),
 ('Film-Noir', 3.9136363636363636),
 ('(no genres listed)', 3.0714285714285716)]

In [ ]:
genreRatingsAverage.saveAsTextFile(output_path)

<span style="color:teal; font-weight: bold">Always stop the contex to ensure a smooth run next time</span>

In [ ]:
sc.stop()